In [ ]:
#%% TRY TO SAVE DERIVATIVE FROM BIDS DATASET HERE:
from main_meg_qc import make_derivative_meg_qc
config_file_name = 'settings.ini'
raw = make_derivative_meg_qc(config_file_name)
#print(all_metric_jsons[0])
#big_rmse_with_value_all_data['mag']
raw

#USED NONFILTERED RAW DATA

In [ ]:
# #%% TRY TO SAVE DERIVATIVE FROM BIDS DATASET HERE:
# from main_meg_qc import make_derivative_meg_qc
# config_file_name = 'settings.ini'
# raw = make_derivative_meg_qc(config_file_name)
# #print(all_metric_jsons[0])
# #big_rmse_with_value_all_data['mag']
# raw

# #FILTERED

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids
import plotly
import mpld3

from main_meg_qc import make_derivative_meg_qc
from initial_meg_qc import get_all_config_params, sanity_check, initial_processing
from RMSE_meq_qc import RMSE_meg_qc
from PSD_meg_qc import PSD_meg_qc
from Peaks_manual_meg_qc import PP_manual_meg_qc
from Peaks_auto_meg_qc import PP_auto_meg_qc
from ECG_meg_qc import ECG_meg_qc
from EOG_meg_qc import EOG_meg_qc
from universal_html_report import keep_fig_derivs, make_joined_report
import time


In [ ]:
from main_meg_qc import get_all_config_params, initial_processing
all_qc_params=get_all_config_params('settings.ini')
#print(all_qc_params)

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif' #GOOD ECG CHANNEL
#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/mikado/sub_HT05ND16/210811/mikado-1.fif' #NO ECG CHANNEL, GOOD RECONSTRUCT


#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/forrest_gump_meg/en04ns31_vp15/190524/vp15_block1-1.fif'  #BAD ECG CHANNEL, 2 good eog.

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004229/sub-102/meg/sub-102_task-amnoise_meg.fif' #2EOG channels, both bad

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif'
#2EOG channels, both bad

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif'
#normal psd

data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind004/ses-01/meg/sub-mind004_ses-01_task-auditory_meg.fif'
#normal but difficult psd

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif'
#EOG 061 bad (or rather unusual), EOG 062 good. Mne takes only the good channel and calculates events on base of it -  
# my average and other plots are only on base of 1 goodchannel automatically

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-003/ses-01/meg/sub-003_ses-01_task-AversiveLearningReplay_run-01_meg.fif'

#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

raw2 = mne.io.read_raw_fif(data_file, allow_maxshield=True)
raw_cropped = raw2.copy()
tmin_my_plot=200
tmax_my_plot=300
duration_my_plot=tmax_my_plot-tmin_my_plot
raw_cropped.crop(tmin=tmin_my_plot, tmax=tmax_my_plot)

#raw_cropped.drop_channels(ECG_channel_name)

raw_cropped
raw2

In [ ]:
#use it filtered 0.5-100

raw2.crop(tmin=0, tmax=120)
raw2.load_data(verbose=True) #Data has to be loaded into mememory before filtering:
raw2_filtered = raw2.copy()
l_freq=0.5
h_freq=100

raw2_filtered.filter(l_freq=l_freq, h_freq=h_freq, picks='meg', method='iir', iir_params=None)
m_or_g='mag'

psds, freqs = raw2_filtered.compute_psd(method='welch', fmin=l_freq, fmax=h_freq, picks=m_or_g, n_jobs=-1, n_fft=2500, n_per_seg=2500).get_data(return_freqs=True)

raw2_filtered

In [ ]:
from universal_plots import get_tit_and_unit
from scipy.signal import find_peaks, peak_widths
from PSD_meg_qc import make_simple_metric_psd

def split_blended_freqs(noisy_freq_bands_idx, width_heights, freqs):

    band = 0
    while band < len(noisy_freq_bands_idx):

        # Checking if the last element of every band is contained in the current band
        last = 0
        while last < len(noisy_freq_bands_idx):

            if (noisy_freq_bands_idx[last] != noisy_freq_bands_idx[band]) and (noisy_freq_bands_idx[last][-1] in noisy_freq_bands_idx[band]):
                
                #if yes - split the biggest band at the split point and also assign the same heights of peaks to both parts.

                split_index = noisy_freq_bands_idx[band].index(noisy_freq_bands_idx[last][-1])
                #split_index = noisy_freq_bands_idx[last][-1] ???

                split_band_left = noisy_freq_bands_idx[band][:split_index+1]
                split_band_right = noisy_freq_bands_idx[band][split_index+1:]


                noisy_freq_bands_idx[last] = split_band_left
                noisy_freq_bands_idx[band] = split_band_right

                min_width_heights = min(width_heights[last],width_heights[band])
                width_heights[band] = min_width_heights
                width_heights[last] = min_width_heights


                #set both bands to 0, so next time  the check will be done for all the bands from the beginning, 
                # concedering new state of noisy_freq_bands_idx:
                band = 0
                last = 0

            last += 1
        band += 1

    return noisy_freq_bands_idx, width_heights


def find_number_and_power_of_noise_freqs(freqs, psds, helper_plots: bool, m_or_g):

    """
    # 1. Calculate average psd curve over all channels
    # 2. Run peak detection on it -> get number of noise freqs
    # 2*. Split blended freqs
    # 3. Fit curve to the general psd OR cut the noise peaks at the point they start and baseline them to 0.
    # 4. Calculate area under the curve for each noisy peak: area is limited to where amplitude crosses the fitted curve. - count from there."""

    m_or_g_tit, unit = get_tit_and_unit(m_or_g)

    #1.
    avg_psd=np.mean(psds,axis=0)

    #2. 
     
    prominence=(max(avg_psd) - min(avg_psd)) / 50
    peaks, _ = find_peaks(avg_psd, prominence=prominence)
    
    #________
    prominence=(max(avg_psd) - min(avg_psd)) / 50
    peaks_neg, _ = find_peaks(-avg_psd, prominence=prominence)
    peaks_neg=[0]+list(peaks_neg)+[len(avg_psd)-1] #take the first point of psd always as the start of noise-free curve
    
    peaks_neg_cut=[] 
    for p_n in range(0, len(peaks_neg)-1):
        if avg_psd[peaks_neg][p_n+1]<=avg_psd[peaks_neg][p_n]:
            peaks_neg_cut.append(peaks_neg[p_n+1])
    peaks_neg_cut=[0]+peaks_neg_cut



    #fit polynomial models up to degree 5
    fig = go.Figure()
    print('HERE!', freqs[peaks_neg_cut], avg_psd[peaks_neg_cut])
    model1 = np.polyfit(np.log(freqs[peaks_neg_cut]), avg_psd[peaks_neg_cut], 1)
    model2 = np.polyfit(np.log(freqs[peaks_neg_cut]), avg_psd[peaks_neg_cut], 2)
    model3 = np.polyfit(np.log(freqs[peaks_neg_cut]), avg_psd[peaks_neg_cut], 3)
    model4 = np.polyfit(np.log(freqs[peaks_neg_cut]), avg_psd[peaks_neg_cut], 4)
    model5 = np.polyfit(np.log(freqs[peaks_neg_cut]), avg_psd[peaks_neg_cut], 5)

    y = [model1[0]*np.log(freqs)+ model1[1]] 
    print('stuff!!!', y[0])

    fig.add_trace(go.Scatter(x=freqs, y=avg_psd, name='data'))
    fig.add_trace(go.Scatter(x=freqs, y=y[0], name='model1')) 
    #fig.add_trace(go.Scatter(x=freqs, y=model1(freqs), name='model1')) 
    #fig.add_trace(go.Scatter(x=freqs, y=model2(freqs), name='model2')) 
    #fig.add_trace(go.Scatter(x=freqs, y=model3(freqs), name='model3')) 
    #fig.add_trace(go.Scatter(x=freqs, y=model4(freqs), name='model4')) 
    #fig.add_trace(go.Scatter(x=freqs, y=model5(freqs), name='model5')) 

    fig.show()


    #________


    widths, width_heights, left_ips, right_ips = peak_widths(avg_psd, peaks, rel_height=1)

    #Plot signal, peaks and contour lines at which the widths where calculated
    from PSD_meg_qc import Power_of_band
    from universal_plots import plot_pie_chart_freq
    from scipy.integrate import simps

    print('Central Freqs: ', freqs[peaks])
    print('Central Amplitudes: ', avg_psd[peaks])
    print('width_heights: ', width_heights)


    avg_psd_only_signal=avg_psd.copy()
    avg_psd_only_peaks=avg_psd.copy()
    avg_psd_only_peaks[:]=None
    avg_psd_only_peaks_baselined=avg_psd.copy()
    avg_psd_only_peaks_baselined[:]=0

    noisy_freq_bands_idx=[]
    for ip_n, _ in enumerate(peaks):
        #noisy_freq_bands_idx.append([fr for fr in np.arange((round(left_ips[ip_n])), round(right_ips[ip_n]))])

        #+1 here because I  will use these values as range,and range in python is usually "up to the value but not including", this should fix it to the right rang
        noisy_freq_bands_idx.append([fr for fr in np.arange((round(left_ips[ip_n])), round(right_ips[ip_n])+1)])
        if noisy_freq_bands_idx[ip_n][0]==noisy_freq_bands_idx[ip_n-1][-1]:
            noisy_freq_bands_idx[ip_n-1].pop(-1)
        #in case the las  element of one band is the same as first of another band, remove the last  elemnt of previos.So bands dont cross.

    #2*
    print('HERE! BEFORE SPLIT')
    print(noisy_freq_bands_idx)
    noisy_freq_bands_idx_split, width_heights_split = split_blended_freqs(noisy_freq_bands_idx, width_heights, freqs)
    print('HERE! AFTER SPLIT')
    print(noisy_freq_bands_idx_split)


    #3.
    ips_l, ips_r = [], []
    for fr_n, fr_b in enumerate(noisy_freq_bands_idx_split):
        ips_l.append(freqs[fr_b][0])
        ips_r.append(freqs[fr_b][-1])
        
        avg_psd_only_signal[fr_b]=None #keep only main psd, remove noise bands, just for visual
        avg_psd_only_peaks[fr_b]=avg_psd[fr_b].copy() #keep only noise bands, remove psd, again for visual
        avg_psd_only_peaks_baselined[fr_b]=avg_psd[fr_b].copy()-[width_heights_split[fr_n]]*len(avg_psd_only_peaks[fr_b])
        #keep only noise bands and baseline them to 0 (remove the signal which is under the noise line)

        # clip the values to 0 if they are negative, they might appear in the beginning of psd curve, 
        # because the first peak might be above even the higher part of psd. should look intp it? 
        # maybe that pesk should not be seen as peak at all?
        avg_psd_only_peaks_baselined=np.array(avg_psd_only_peaks_baselined) 
        avg_psd_only_peaks_baselined = np.clip(avg_psd_only_peaks_baselined, 0, None) 


    if helper_plots is True:
        import matplotlib.pyplot as plt

        fig, axs = plt.subplots(2, 3, figsize=(13, 8))

        axs[0, 0].plot(freqs,avg_psd)
        axs[0, 0].plot(freqs[peaks], avg_psd[peaks], 'x')
        axs[0, 0].plot(freqs[peaks_neg], avg_psd[peaks_neg], 'o')
        xmin_f=[round(l) for l in left_ips]
        xmax_f=[round(r) for r in right_ips]
        xmin=[freqs[i] for i in xmin_f]
        xmax=[freqs[i] for i in xmax_f]
        axs[0, 0].hlines(y=width_heights, xmin=xmin, xmax=xmax, color="C3")
        axs[0, 0].set_title('1. PSD Welch with peaks, blended freqs not split yet. \n Shown as detected by peak_widths')
        axs[0, 0].set_xlim(freqs[0], freqs[-1])
        axs[0, 0].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)

        axs[0, 1].plot(freqs,avg_psd)
        axs[0, 1].plot(freqs[peaks], avg_psd[peaks], 'x')
        axs[0, 1].plot(freqs[peaks_neg_cut], avg_psd[peaks_neg_cut], 'o')
        xmin_f=[round(l) for l in left_ips]
        xmax_f=[round(r) for r in right_ips]
        xmin=[freqs[i] for i in xmin_f]
        xmax=[freqs[i] for i in xmax_f]
        axs[0, 1].hlines(y=width_heights, xmin=xmin, xmax=xmax, color="C3")
        axs[0, 1].set_title('1. PSD Welch with peaks, blended freqs not split yet. \n Shown as detected by peak_widths')
        axs[0, 1].set_xlim(freqs[0], freqs[-1])
        axs[0, 1].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)

        axs[1, 1].plot(freqs,-avg_psd)
        axs[1, 1].plot(freqs[peaks], -avg_psd[peaks], 'x')
        axs[1, 1].plot(freqs[peaks_neg], -avg_psd[peaks_neg], 'o')
        xmin_f=[round(l) for l in left_ips]
        xmax_f=[round(r) for r in right_ips]
        xmin=[freqs[i] for i in xmin_f]
        xmax=[freqs[i] for i in xmax_f]
        axs[1, 1].hlines(y=width_heights, xmin=xmin, xmax=xmax, color="C3")
        axs[1, 1].set_title('1. PSD Welch with peaks, blended freqs not split yet. \n Shown as detected by peak_widths')
        axs[1, 1].set_xlim(freqs[0], freqs[-1])
        #axs[1, 1].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)


        # axs[0, 1].plot(freqs,avg_psd_only_signal)
        # axs[0, 1].plot(freqs[peaks], avg_psd_only_signal[peaks], "x")
        # axs[0, 1].hlines(y=width_heights, xmin=ips_l, xmax=ips_r, color="C3")
        # axs[0, 1].set_title('2. PSD without noise, split blended freqs')
        # axs[0, 1].set_xlim(freqs[0], freqs[-1])
        # axs[0, 1].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)

        axs[1, 0].plot(freqs,avg_psd_only_peaks)
        axs[1, 0].plot(freqs[peaks], avg_psd_only_peaks[peaks], "x")
        axs[1, 0].hlines(y=width_heights, xmin=ips_l, xmax=ips_r, color="C3")
        axs[1, 0].set_title('3. Only noise peaks, split blended freqs')
        axs[1, 0].set_xlim(freqs[0], freqs[-1])
        axs[1, 0].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)

        # axs[1, 1].plot(freqs,avg_psd_only_peaks_baselined)
        # axs[1, 1].plot(freqs[peaks], avg_psd_only_peaks_baselined[peaks], "x")
        # axs[1, 1].hlines(y=[0]*len(freqs[peaks]), xmin=ips_l, xmax=ips_r, color="C3")
        # axs[1, 1].set_title('4. Noise peaks brought to basline, split blended freqs')
        # axs[1, 1].set_xlim(freqs[0], freqs[-1])
        # axs[1, 1].set_ylim(min(avg_psd)*-1.05, max(avg_psd)*1.05)

        plt.tight_layout()
        fig.show()


    #4.
    freq_res = freqs[1] - freqs[0]
    total_power = simps(avg_psd, dx=freq_res) # power of all signal
    print('Total power: ', total_power)

    all_bp_noise=[]
    all_bp_relative=[]
    bp_noise_relative_to_signal=[]

    avg_psd_only_peaks_baselined_new=np.array([avg_psd_only_peaks_baselined]) 

    for fr_n, fr_b in enumerate(noisy_freq_bands_idx_split):

        #print('band',  freqs[fr_b][0], freqs[fr_b][-1])
        bp_noise, _, bp_relative = Power_of_band(freqs=freqs, f_low = freqs[fr_b][0], f_high= freqs[fr_b][-1], psds=avg_psd_only_peaks_baselined_new)

        all_bp_noise+=bp_noise
        all_bp_relative+=bp_relative

        #Calculate how much of the total power of the average signal goes into each of the noise freqs:
        bp_noise_relative_to_signal.append(bp_noise / total_power) # relative power: % of this band in the total bands power for this channel:

    bp_noise_relative_to_signal=[r[0] for r in bp_noise_relative_to_signal]

    #print('Freq band for each peak:', ips_pair)
    print('BP', all_bp_noise)
    print('relative BP', all_bp_relative)
    print('Amount of noisy freq in total signal', bp_noise_relative_to_signal)


    #Legend for the pie chart:
    bands_legend=[]
    for fr_n, fr in enumerate(freqs[peaks]):
        bands_legend.append(str(fr)+' Hz noise: '+str(all_bp_noise[fr_n])+' '+unit)
    main_signal_legend='Main signal: '+str(total_power-sum(all_bp_noise))+' '+unit
    bands_legend.append(main_signal_legend)
    #bands_legend=[str(fr)+' Hz noise' for fr in freqs[peaks]]+['Main signal'] #legend version without showing the abs power

    Snr=bp_noise_relative_to_signal+[1-sum(bp_noise_relative_to_signal)]
    noise_pie_derivative = plot_pie_chart_freq(mean_relative_freq=Snr, tit='Signal and Noise. '+m_or_g_tit, bands_names=bands_legend)
    noise_pie_derivative.content.show()

    simple_metric_deriv=make_simple_metric_psd(all_bp_noise, bp_noise_relative_to_signal, m_or_g, freqs, peaks)

    return noise_pie_derivative, simple_metric_deriv

In [ ]:
#from PSD_meg_qc import find_number_and_power_of_noise_freqs

noise_pie_derivative, simple_metric_deriv = find_number_and_power_of_noise_freqs(freqs, psds, True, m_or_g)


    

In [ ]:
peaks_neg=np.ndarray(3,   5,   8,  11,  13,  16,  27, 116)
print(peaks_neg)
np.insert(peaks_neg, 0, 0)

In [ ]:
#plot EOG channel

raw.copy().pick_types(meg=False, stim=False,eog=True).plot()

In [ ]:
from ECG_meg_qc import find_affected_channels

mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mag': mag_ch_names, 'grad': grad_ch_names}
sfreq=raw.info['sfreq']
m_or_g='mag'
tmin=-0.2
tmax=0.2
#ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, picks=channels[m_or_g], tmin=tmin, tmax=tmax)

#ch_name='EOG 061'
ch_name='EOG002'
eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], ch_name=ch_name, tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:
ch_name='EOG003'
eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], ch_name=ch_name, tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:

eog_epochs = mne.preprocessing.create_eog_epochs(raw, picks=channels[m_or_g], tmin=tmin, tmax=tmax)


ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(eog_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='EOG', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True,  use_abs_of_all_data='flip')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='ecg', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='False')


In [ ]:
ecg_affected_channels, fig_affected, fig_not_affected, fig_avg=find_affected_channels(ecg_epochs, channels, m_or_g, norm_lvl=1, ecg_or_eog='ecg', thresh_lvl_peakfinder=5, sfreq=sfreq, tmin=tmin, tmax=tmax, plotflag=True, use_abs_of_all_data='flip')


In [ ]:
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003682/sub-001/ses-01/meg/sub-001_ses-01_task-AversiveLearningReplay_run-01_meg.fif'
raw = mne.io.read_raw_fif(data_file)
raw.info['chs']

In [ ]:


import numpy as np
import mne
y=np.array([1,4,5,0,0,0])
peaks_l, peaks_m = mne.preprocessing.peak_finder(y)

print(peaks_l)
print(type(peaks_l[0]))
print(len(peaks_l))

print(np.array([87]))
print(type(np.array([87])[0]))
#print(len(np.array(87)))


print(np.ndarray(87))
print(len(np.ndarray(87)))

In [ ]:
# Other useful ancp stuff:

config = configparser.ConfigParser()
config.read('settings.ini')

default_direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(default_direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()


list_of_entities = layout.get_entities()
print(list_of_entities)

In [ ]:
#TRY SEPARATE FUNCS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
all_qc_params = get_all_config_params('settings.ini')
dict_of_dfs_epoch, dict_epochs_mg, channels, raw_filtered, raw_filtered_resampled, raw_cropped, raw, active_shielding_used = initial_processing(default_settings=all_qc_params['default'], filtering_settings=all_qc_params['Filtering'], epoching_params=all_qc_params['Epoching'], data_file=data_file)

m_or_g_chosen = ['mag']
sid='009'


In [ ]:
start_time = time.time()


#dfs_ptp_amlitude_annot, bad_channels, amplit_annot_with_ch_names = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

#derivs, big_rmse_with_value_all_data, small_rmse_with_value_all_data = RMSE_meg_qc(all_qc_params['RMSE'], channels, dict_epochs_mg, dict_dict_of_dfs_epoch, raw, m_or_g_chosen)

derivs = PP_manual_meg_qc(all_qc_params['PTP_manual'], channels, dict_epochs_mg, dict_of_dfs_epoch, raw, m_or_g_chosen)

#out_with_name_and_format = PSD_meg_qc(sid, config, channels, raw, m_or_g_chosen)

derivs = ECG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format = EOG_meg_qc(config, raw, m_or_g_chosen)

#out_with_name_and_format, bad_channels = PP_auto_meg_qc(sid, config, channels, raw, m_or_g_chosen)

print("--- Execution %s seconds ---" % (time.time() - start_time))


for der in derivs:
    if der.content_type == 'plotly':
        der.content.show()


In [ ]:
print(figs)
all_fig_derivs = figs
figures_report = {}
for x in range(0, len(all_fig_derivs)):
    if all_fig_derivs[x][3]=='plotly':
        figures_report["f{0}".format(x)] = plotly.io.to_html(all_fig_derivs[x][0])
    elif all_fig_derivs[x][3]=='matplotlib':
        figures_report["f{0}".format(x)] = mpld3.fig_to_html(all_fig_derivs[x][0]);

print(figures_report)

In [ ]:
html_string = '''
<!doctype html>
<html>
    <head>
        <meta charset="UTF-8">
        <title>MEG QC: Frequency spectrum Report</title>
        <style>body{ margin:0 100;}</style>
    </head>
    
    <body style="font-family: Arial">
        <center>
        <h1>MEG data quality analysis report</h1>
        <br></br>
        <!-- *** Section 1 *** --->
        <h2>Frequency spectrum per channel</h2>
        ''' + html_fig + '''
        <p>graph description...</p>
        </center>
    
    </body>
</html>'''

with open('report_trial.html', 'w', encoding = 'utf8') as f:
    f.write(html_string)

In [ ]:
#config = configparser.ConfigParser()
#config.read('settings.ini')

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds000117/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_meg.fif'
# file does not start with a file id tag

#data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003392/sub-01/meg/sub-01_task-localizer_meg.fif'
# SSS frilter. need to allow maxshiled.

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003694/sub-01/meg/sub-01_task-MEM_run-01_meg.fif'
#raw = mne.io.read_raw_fif(data_file, on_split_missing='ignore')


#dict_of_dfs_epoch, dict_epochs_mg, channels, raw_bandpass, raw_filtered_resampled, raw_cropped, raw = initial_processing(config, data_file)

#data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003922/sub-Mp150285/ses-01/meg/sub-Mp150285_ses-01_acq-crosstalk_meg.fif'
#Could not find measurement data: how many in this set, which subjects?


data_file ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds004229/sub-102/meg/sub-102_task-amnoise_meg.fif'
#SSS filter
raw = mne.io.read_raw_fif(data_file, allow_maxshield=True)


#data_dir ='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/not fitting no fif/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
#raw = mne.io.read_raw_ctf(data_dir)
